In [1]:
import helper
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as offline_py
offline_py.init_notebook_mode(connected=True)
import os


In [2]:
def _generate_stock_trace(prices):
    return go.Scatter(
        name='Index',
        x=prices.index,
        y=prices,
        line={'color': helper.color_scheme['major_line']})


def _generate_traces(name_df_color_data):
    traces = []

    for name, df, color in name_df_color_data:
        traces.append(go.Scatter(
            name=name,
            x=df.index,
            y=df,
            mode='line',
            line={'color': color}))

    return traces

In [3]:
def daily_returns(df):
    daily_returns = df.copy()
    daily_returns[1:] = (df[1:] / df[:-1].values) - 1
    daily_returns.iloc[0,:] = 0
    return daily_returns

In [4]:
def fill_missing_values(df_data):
    """Fill missing values in data frame, in place."""
    ##########################################################
    df_data.fillna(method='ffill',inplace=True)
    df_data.fillna(method='bfill',inplace=True)

In [5]:
def symbol_to_path(symbol, base_dir="data"):
    """Return CSV file path given ticker symbol."""
    return os.path.join(base_dir, "{}.csv".format(str(symbol)))

In [6]:
def get_data(symbols, dates):
    """Read stock data (adjusted close) for given symbols from CSV files."""
    df_final = pd.DataFrame(index=dates)
    if "SPY" not in symbols:  # add SPY for reference, if absent
        symbols.insert(0, "SPY")
        
    for symbol in symbols:
        file_path = symbol_to_path(symbol)
        df_temp = pd.read_csv(file_path, parse_dates=True, index_col="Date",
            usecols=["Date", "Adj Close"], na_values=["nan"])
        df_temp = df_temp.rename(columns={"Adj Close": symbol})
        df_final = df_final.join(df_temp)
        if symbol == "SPY":  # drop dates SPY did not trade
            df_final = df_final.dropna(subset=["SPY"])
    fill_missing_values(df_final)
    return df_final

In [7]:
def plot_data(df_data,title = 'Stock'):
    """Plot stock data with appropriate axis labels."""
    config = helper.generate_config()
    layout = go.Layout(title="{} Data".format(title))
    stock_trace = _generate_stock_trace(df_data)
    offline_py.iplot({'data' : [stock_trace], 'layout': layout}, config=config)

In [8]:
def get_rolling_mean(values, window):
    """Return rolling mean of given values, using specified window size."""
    return values.rolling(window).mean()


def get_rolling_std(values, window):
    """Return rolling standard deviation of given values, using specified window size."""
    return values.rolling(window).std()
    # TODO: Compute and return rolling standard deviation


def get_bollinger_bands(rm, rstd):
    """Return upper and lower Bollinger Bands."""
    # TODO: Compute upper_band and lower_band
    upper_band = rm + rstd*2
    lower_band = rm - rstd*2
    return upper_band, lower_band

In [9]:
def plot_bollinger_bands(df_data,symbol,window=20):
    # Compute rolling mean
    rm = get_rolling_mean(df_data[symbol],window)
    
    #Compute std deviation
    rstd = get_rolling_std(df_data[symbol],window)
    
    #Compute Bollinger Bands
    upper_band,lower_band = get_bollinger_bands(rm,rstd)
    
    #Plots
    ## plot raw 'symbol' data followed by mean and bollinger bands
    config = helper.generate_config()
    layout = go.Layout(title='Bollinger')
    traces = _generate_traces([
        ('Prices', df_data[symbol], helper.color_scheme['index']),
        ('Rolling Mean', rm, helper.color_scheme['mean']),
        ('Upper band', upper_band, helper.color_scheme['green']),
        ('Lower band', lower_band, helper.color_scheme['red']),
    ])
    
    offline_py.iplot({'data' : traces, 'layout': layout}, config=config)